In [ ]:
"use client"

import React, { useEffect, useState } from "react"
import axios from "axios"
import { useForm } from "react-hook-form"
import { Plus, Minus, Trash2, ShoppingCart, Loader2 } from "lucide-react"
import { toast } from "sonner"

import { Button } from "@/components/ui/button"
import {
  Sheet,
  SheetContent,
  SheetDescription,
  SheetFooter,
  SheetHeader,
  SheetTitle,
  SheetTrigger,
} from "@/components/ui/sheet"
import { Separator } from "@/components/ui/separator"
import { Avatar, AvatarFallback, AvatarImage } from "@/components/ui/avatar"
import { Input } from "@/components/ui/input"
import { Label } from "@/components/ui/label"

interface CartItem {
  id: number
  product: {
    id: number
    name: string
    price: number
    image: string
  }
  quantity: number
  total_price: number
}

interface CartData {
  items: CartItem[]
  total_price: number
  total_items: number
}

export default function CartSheet() {
  const [isOpen, setIsOpen] = useState(false)
  const [cart, setCart] = useState<CartData | null>(null)
  const [loading, setLoading] = useState(true)
  const [updating, setUpdating] = useState<number | null>(null)

  const fetchCart = async () => {
    setLoading(true)
    try {
      const response = await axios.get('/api/cart/')
      setCart(response.data)
    } catch (error) {
      console.error('Error fetching cart:', error)
      toast.error('Failed to load cart data')
    } finally {
      setLoading(false)
    }
  }

  useEffect(() => {
    if (isOpen) {
      fetchCart()
    }
  }, [isOpen])

  const updateQuantity = async (itemId: number, newQuantity: number) => {
    if (newQuantity < 1) return
    
    setUpdating(itemId)
    try {
      await axios.put(`/api/cart/${itemId}/`, {
        quantity: newQuantity
      })
      await fetchCart()
      toast.success('Cart updated')
    } catch (error) {
      console.error('Error updating cart:', error)
      toast.error('Failed to update cart')
    } finally {
      setUpdating(null)
    }
  }

  const removeItem = async (itemId: number) => {
    setUpdating(itemId)
    try {
      await axios.delete(`/api/cart/${itemId}/`)
      await fetchCart()
      toast.success('Item removed from cart')
    } catch (error) {
      console.error('Error removing item:', error)
      toast.error('Failed to remove item')
    } finally {
      setUpdating(null)
    }
  }

  const handleCheckout = () => {